# Customer Retention Classifier

In [348]:
import math
import re
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import Imputer
from scipy.stats import mode

In [349]:
pd.set_option("display.max_columns", 100)
pd.options.display.max_columns

100

In [350]:
TRAIN_DATA = pd.read_csv("../../../kaggle_data/customer_retention/midterm_train.csv")
TEST_DATA = pd.read_csv("../../../kaggle_data/customer_retention/midterm_test.csv")
X = TRAIN_DATA
X_combined = pd.concat([X, TEST_DATA])

In [351]:
X_combined.shape

(200000, 51)

### Clean Month Data

In [352]:
def cleanMonth(x):
    if isinstance(x, str):
        if x == 'Jun': return 'june'
        elif x == 'July': return 'july'
        elif x == 'Aug': return 'august'
        elif x == 'May': return 'may'
        elif x == 'Mar': return 'march'
        elif x == 'Apr': return 'april'
        elif x == 'sept.': return 'september'
        elif x == 'Feb': return 'february'
        elif x == 'Oct': return 'october'
        elif x == 'Nov': return 'november'
        elif x == 'January': return 'january'
        elif x == 'Dev': return 'december'

month = [cleanMonth(x) for x in X_combined.x19]

### Clean Weekday Data

In [353]:
def cleanWeekday(x):
    if isinstance(x, str):
        if x == 'monday': return 'monday'
        elif x == 'tuesday': return 'tuesday'
        elif x == 'wednesday': return 'wednesday'
        elif x == 'thurday': return 'thursday'
        elif x == 'friday': return 'friday'

weekday = [cleanWeekday(x) for x in X_combined.x43]

### Clean Region Data

In [354]:
def cleanRegion(x):
    if isinstance(x, str):
        if x == 'asia': return 'asia'
        elif x == 'euorpe': return 'europe'
        elif x == 'america': return 'america'

region = [cleanRegion(x) for x in X_combined.x16]

### Parse Money to Float

In [355]:
def parseMoneyToFloat(x):
    if isinstance(x, str): return float(re.sub('[$,()]', '', x))

X_combined.x44 = [parseMoneyToFloat(x) for x in X_combined.x44]

### Parse Percent to Float

In [356]:
def parsePercentToFloat(x):
    if isinstance(x, str): return float(re.sub('%', '', x)) / 100.00

X_combined.x09 = [parsePercentToFloat(x) for x in X_combined.x09]

### Imputation  Strategies

In [357]:
X_combined.shape

(200000, 51)

1. **df0**: Numerical: Drop, Categorical: Drop
2. **df1**: Numerical: Mean, Categorical: Frequency
3. **df2**: Numerical: Median, Categorical: Frequency
4. **df3**: k-Nearest Neighbors
5. **df4**: Random Forest

In [358]:
X_numeric = X_combined.copy()
X_numeric.drop(['x16', 'x19', 'x43'], axis=1, inplace=True)

In [359]:
df0 = X_numeric.copy()
df1 = X_numeric.copy()
df2 = X_numeric.copy()

### Impute Missing Categorical Data with Most Frequent Value

In [360]:
X_categorical = pd.DataFrame.from_dict({ 'month': month, 'weekday': weekday, 'region': region })

In [361]:
X_categorical.month.value_counts()[[0]]

june    55795
Name: month, dtype: int64

In [362]:
X_categorical.weekday.value_counts()[[0]]

wednesday    126413
Name: weekday, dtype: int64

In [363]:
X_categorical.region.value_counts()[[0]]

asia    173129
Name: region, dtype: int64

In [364]:
X_categorical.month.fillna('june', inplace=True)
X_categorical.weekday.fillna('wednesday', inplace=True)
X_categorical.region.fillna('asia', inplace=True)

### df0: Numerical: Drop, Categorical: Drop

In [365]:
df0.dropna(inplace=True, subset=df0.columns[0:-1])
df0.shape

(198116, 48)

### df1: Numerical: Mean, Categorical: Most Frequent

In [366]:
y_df1 = df1.pop('y')

In [367]:
meanImputer = Imputer(strategy='mean', copy=False)
meanImputer.fit_transform(df1)

array([[ -5.90070664e+00,   4.69388657e-01,   4.73306785e+00, ...,
          4.59280348e+00,  -3.44776760e-02,   3.68040134e+00],
       [ -3.53039065e+00,   4.22981797e+00,  -4.61943288e+00, ...,
         -5.46246671e+00,  -2.29517509e-01,  -2.33294711e+00],
       [  1.00338855e+00,  -1.25474570e-02,   6.25050323e+00, ...,
         -1.14070719e+01,   6.39555117e+00,   3.54533178e+00],
       ..., 
       [  1.71043472e+01,   1.08815589e+01,   9.25710482e+00, ...,
          6.07498027e+00,  -2.24617902e-01,   5.16245808e+00],
       [  1.03181728e+01,  -8.79384077e+00,  -2.67099784e+00, ...,
         -7.78997798e+00,   1.60228442e-01,  -2.71319396e+00],
       [ -3.75580866e+00,   6.22956464e+00,  -1.28682043e+00, ...,
          6.10818001e+01,   1.34704559e+00,  -7.67326474e-01]])

In [368]:
df1['y'] = y_df1

### df2: Numerical: Median, Categorical: Most Frequent

In [369]:
y_df2 = df2.pop('y')

In [370]:
medianImputer = Imputer(strategy='median', copy=False)
medianImputer.fit_transform(df2)

array([[ -5.90070664e+00,   4.69388657e-01,   4.73306785e+00, ...,
          4.59280348e+00,  -3.44776760e-02,   3.68040134e+00],
       [ -3.53039065e+00,   4.22981797e+00,  -4.61943288e+00, ...,
         -5.46246671e+00,  -2.29517509e-01,  -2.33294711e+00],
       [  1.00338855e+00,  -1.25474570e-02,   6.25050323e+00, ...,
         -1.14070719e+01,   6.39555117e+00,   3.54533178e+00],
       ..., 
       [  1.71043472e+01,   1.08815589e+01,   9.25710482e+00, ...,
          6.07498027e+00,  -2.24617902e-01,   5.16245808e+00],
       [  1.03181728e+01,  -8.79384077e+00,  -2.67099784e+00, ...,
         -7.78997798e+00,   1.60228442e-01,  -2.71319396e+00],
       [ -3.75580866e+00,   6.22956464e+00,  -1.28682043e+00, ...,
          6.10818001e+01,   1.34704559e+00,  -7.67326474e-01]])

In [371]:
df2['y'] = y_df2

### Convert Categorical Variables to Dummies & Append to Numeric Dataframes

In [372]:
X_dummies = pd.get_dummies(X_categorical)

In [373]:
df0

x00        x01        x02        x03        x04       x05  \
0      -5.900707   0.469389   4.733068   9.223027   3.170787  0.024669   
1      -3.530391   4.229818  -4.619433   2.154913   5.619903 -1.955667   
2       1.003389  -0.012547   6.250503   3.368675   1.711321  6.192718   
3     -16.041769  -3.243507  -0.546649 -11.594427   5.111965  0.609785   
4      20.498575  10.766191   1.478032  -5.723594  -0.768839 -3.482515   
5      -0.830855  -4.026283  -2.961508 -14.230334  -4.052836  0.486097   
6     -25.664561   9.442661   4.179044  -6.127033   9.789203  2.792734   
7      14.727019  -4.769810   0.545849 -14.474038   3.551106  0.135631   
8      11.340836   1.756355   3.809453   9.021020  10.804382 -1.448350   
9      15.281702   4.187856  -1.403868  -1.072465   0.804286 -2.001035   
10     22.553588  20.194411  -2.280246   5.773427  10.228747  2.916477   
11      2.840066   4.485118  -4.137530   8.348743   2.043830 -1.071960   
12      4.750975   0.441782   5.884518  -6.213406  -6.679617  4.224643   
13    -13.472503  -2.173725  -9.576501 -15.148966   2.692548  0.027501   
14    -17.055826   3.909502  11.239432   7.794344 -10.084874 -2.914232   
15     16.860151   1.401590   0.656391   6.127848   6.910910 -1.155880   
16    -13.373212   3.140720  -2.774864  -4.822319  -2.002955 -3.544629   
17     14.122019   8.806063   1.165677   5.488228  -0.232794  2.303123   
18    -23.690343 -12.673660   0.246888  20.568391   6.178834 -3.144562   
19     -4.575471   9.161018   0.057567 -11.957873  -8.355703  3.788993   
20     34.330689   4.701084  -5.020014  -0.515706  -1.351645  1.579211   
21     25.572405   5.416426   1.051341  11.108177   4.965679  0.418946   
22      9.667193   7.791684  -2.516034   9.264247  -3.800270 -1.132911   
23     27.373386  12.816783  -0.922720   4.523277   5.074994 -2.587429   
24      9.090015   8.900854   5.240586   3.216846   6.479336 -1.344419   
25     30.937273 -15.167300  -5.560264   5.372044   8.582052 -2.361488   
26      0.088776  -1.337917   1.464665   3.095267   2.721295  2.132710   
27     13.790713  -9.049010   4.792523  -3.057832  -1.429218 -1.180864   
28     -7.035771  -4.197586  12.936235   1.468682   3.989373  2.434248   
29    -27.316709  26.556007   9.072305 -23.491250   2.010969  5.960815   
...          ...        ...        ...        ...        ...       ...   
39969 -18.395325   1.509594   0.359782 -14.031736 -11.424541 -1.093255   
39970  37.963100 -12.256992  -3.038262  10.224146  -0.021389 -2.906670   
39971  11.585941  10.967134 -16.854543   9.797261  -1.668845  0.407933   
39972   1.407500   4.497659  -7.618577 -10.821665   5.035274  1.607229   
39973 -29.475440   4.380808   5.827111   6.379580 -10.953355  2.367653   
39974 -22.505088  -3.817627  -3.235677  17.138335 -10.160060  0.123469   
39975  -6.511789  -5.988581   7.378607  16.029424  -3.855702  0.620857   
39976  -5.085414   6.450312  -3.465655  -2.911150   1.473337 -3.701404   
39977   5.488057   4.292509   1.348368   2.227747   9.406587  0.286637   
39978 -21.215113  -0.398796  -2.334645   5.636091  -8.027192 -0.381604   
39979  28.649986 -10.167627   2.814307   3.586369   3.254008 -2.558978   
39980   5.978844   2.361925  -4.419037  -5.053635   4.589658 -1.787473   
39981 -26.750652  -9.135328  -6.255429   8.036704   7.923440  0.878867   
39982  -5.287918   6.120939   5.723453  -5.065494  -5.473237 -0.582789   
39983   6.571969  -3.727379 -13.405899   4.889092   2.863514  0.437106   
39984 -15.251670   2.055345  -8.883414  -1.793493 -10.179739  2.159858   
39985 -14.127031 -12.745440   4.015829  15.107972   1.548051 -3.313054   
39986 -21.943914   5.158877   1.285645  -4.447155   3.756907  4.101585   
39987   5.260151   6.604971  -9.771529   1.251739  14.495470  0.944889   
39988 -19.130700  -5.381221  14.663944   5.921802   0.311157  0.206562   
39989 -10.114293   2.395771 -11.439071   0.803629   3.185572  1.476706   
39990  12.643741  -3.908170   3.101587   9.281820   5.067780 -2.584130   
39991  10.082824 